In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import diversity_tools as dt
from pyproj import Geod

In [15]:
selected_cbg_brand = pd.read_csv('selected_cbg_brand.csv', index_col = 'cbg')
selected_brands = selected_cbg_brand.columns.values
selected_cbg = selected_cbg_brand.index.values
NY_brands = pd.read_csv('NY_brands.csv')
cbg_unpack = pd.read_csv('cbg_unpack.csv', dtype = {'visitor_home_cbgs': str})
cbg_geo = pd.read_csv('../SafeGraph_data/safegraph_open_census_data_2019/metadata/cbg_geographic_data.csv', 
                     usecols = ['census_block_group','latitude', 'longitude'],
                     dtype = {'census_block_group' : 'str'})
selected_cbg_income = pd.read_csv('selected_cbg_income.csv', index_col = 'cbg')

# get availability

In [3]:
def get_availability_matrix():
    # just get the brands and their home cbg
    df = NY_brands[['brands', 'poi_cbg']]
    df = df.astype({'poi_cbg': 'int'})
    # select only the brands and cbg in our sample
    df = df[df['brands'].isin(selected_brands) & df['poi_cbg'].isin(selected_cbg)]
    # unstack
    availability_matrix = df.groupby(["poi_cbg", 'brands']).size().unstack()
    availability_matrix = availability_matrix.fillna(0)
    return availability_matrix

In [4]:
availability_matrix = get_availability_matrix()

In [5]:
availability_matrix

brands,& Other Stories,&pizza,1-800-Flowers,10 Spot/MadRag,110 Grill,16 Handles,24 Hour Fitness,2nd Wind Exercise Equipment,5 Napkin Burger,5.11 Tactical,...,iFly Indoor Skydiving,iFresh Market,iLoveKickboxing,itsu,junzi,lululemon athletica,maurices,rue21,sweetgreen,vineyard vines
poi_cbg,,,,,,,,,,,,,,,,,,,,,
360010002001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
360010002002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
360010003001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
360010003002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
360010003003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361231501005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361231502003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361231504004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# save for separate regression analysis by industry 
# availability_matrix.to_csv('availability_matrix.csv') 

In [7]:
cbg_availability = dt.get_3diversity(availability_matrix)

In [8]:
cbg_availability

,entropy_brand,brand_std,entropy_price
cbg,,,
360010002001,0.000000,NaN,0.000000
360010002002,0.000000,NaN,0.000000
360010003001,0.383831,12264.087267,0.445246
360010003002,0.000000,NaN,0.000000
360010003003,0.101716,6293.957459,0.500000
...,...,...,...
361231501005,0.203432,7352.195794,0.000000
361231502003,0.161216,5448.674709,0.459148
361231504004,0.161216,6280.507066,0.459148


In [9]:
cbg_availability.isna().sum()

entropy_brand       0
brand_std        2430
entropy_price     353
dtype: int64

In [36]:
len(cbg_availability.dropna(subset=['brand_std']))

3706

In [37]:
len(cbg_availability.dropna(subset=['entropy_price']))

5783

In [10]:
# cbg_availability.to_csv('cbg_availability.csv')

In [16]:
income_availability = pd.merge(cbg_availability, selected_cbg_income, left_index = True, right_index = True)

In [19]:
dt.corr_table(income_availability)

,entropy_brand,brand_std,entropy_price,income
entropy_brand,1.0***,0.192***,0.708***,0.075***
brand_std,0.192***,1.0***,0.162***,0.201***
entropy_price,0.708***,0.162***,1.0***,0.082***
income,0.075***,0.201***,0.082***,1.0***


# get mobility

In [21]:
def get_geo_df():
    # get placekey, latitude and longitude 
    poi_geo = NY_brands[['placekey', 'latitude', 'longitude']]
    # only use the select brands and cbgs
    selected_unpack = cbg_unpack[cbg_unpack['brands'].isin(selected_brands) & 
                                cbg_unpack['visitor_home_cbgs'].isin(selected_cbg.astype(str))]
    geo_df = selected_unpack.merge(poi_geo, on = 'placekey')
    geo_df = geo_df.merge(cbg_geo, left_on = 'visitor_home_cbgs', right_on = 'census_block_group')
    geo_df = geo_df.drop(columns = ['placekey', 'census_block_group'])
    # get the distance km https://stackoverflow.com/a/44446971/12148092
    wgs84_geod = Geod(ellps='WGS84')
    az12, az21, dist = wgs84_geod.inv(geo_df['longitude_x'], geo_df['latitude_x'],
                                    geo_df['longitude_y'], geo_df['latitude_y'])
    geo_df = geo_df.assign(distance = dist/1000)
    # only select useful columns
    geo_df = geo_df[['brands', 'visitor_home_cbgs', 'cbg_visitor_count', 'distance']]
    geo_df = geo_df.assign(sum_dist = geo_df['cbg_visitor_count'] * geo_df['distance'])
    geo_df = geo_df.drop(columns = ['distance'])

    return geo_df


In [22]:
geo_df = get_geo_df()

In [23]:
geo_df

,brands,visitor_home_cbgs,cbg_visitor_count,sum_dist
0,CVS,360810717013,4,43.372952
1,Mobil,360810717013,8,1.181426
2,Costco,360810717013,5,1.527683
3,Red Mango,360810717013,4,58.303806
4,McDonald's,360810717013,4,2.908930
...,...,...,...,...
1042448,McDonald's,360239704003,4,142.186633
1042449,Tops Friendly Markets,360239704003,9,41.598864
1042450,Burger King,360239704003,4,18.765262
1042451,Walgreens,360239704003,4,11.243144


In [24]:
# save for separate analysis by industry
# geo_df.to_csv('geo_df.csv', index = False)

In [25]:
cbg_mobility = dt.get_mobility(geo_df)
# cbg_mobility.to_csv('cbg_mobility.csv')

In [29]:
cbg_mobility.index = cbg_mobility.index.astype(int)
income_mobility = pd.merge(cbg_mobility, selected_cbg_income, left_index = True, right_index = True)

In [34]:
income_mobility

,mobility,income
cbg,,
100030145022,216.637613,20833.0
120110105032,1705.825095,200099.0
120950171031,1549.765987,75404.0
120950171032,1584.010885,79577.0
120970408021,1570.592052,59140.0
...,...,...
421179504001,34.459485,38646.0
421179505003,40.371778,48625.0
421179506002,73.223299,36042.0


In [32]:
stats.pearsonr(income_mobility['mobility'], income_mobility['income'])

(-0.0035585097549508298, 0.6779752121145481)

In [33]:
dt.corr_table(income_mobility)

,mobility,income
mobility,1.0***,-0.004
income,-0.004,1.0***
